<a href="https://colab.research.google.com/github/Sujana2004/VoyageSense/blob/main/VoyageSenseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # ============================================
# # FIXED: OLLAMA WITH OPENAI COMPATIBILITY
# # ============================================

# import subprocess
# import threading
# import time
# import re
# import os

# print("🚀 Starting Ollama with OpenAI Compatibility Mode...")
# print("="*60)

# # 1. Install Ollama
# print("\n📦 Installing Ollama...")
# !curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1
# print("✅ Ollama installed!")

# # 2. Start Ollama with OpenAI compatibility
# print("\n🔧 Starting Ollama server...")

# # Set environment for OpenAI compatibility
# os.environ['OLLAMA_ORIGINS'] = '*'
# os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'

# def run_ollama():
#     env = os.environ.copy()
#     env.update({
#         'OLLAMA_ORIGINS': '*',
#         'OLLAMA_HOST': '0.0.0.0:11434'
#     })
#     subprocess.run(['ollama', 'serve'], env=env,
#                    stdout=subprocess.DEVNULL,
#                    stderr=subprocess.DEVNULL)

# ollama_thread = threading.Thread(target=run_ollama, daemon=True)
# ollama_thread.start()
# time.sleep(7)  # Give more time to start

# # 3. Verify Ollama is running
# print("🔍 Verifying Ollama...")
# result = subprocess.run(['curl', '-s', 'http://localhost:11434'],
#                        capture_output=True, text=True)
# if "Ollama" in result.stdout or result.returncode == 0:
#     print("✅ Ollama is running!")
# else:
#     print("⚠️ Ollama might still be starting...")

# !ollama list

# # 4. Pull model
# print("\n📥 Downloading model (2-5 minutes)...")
# model_result = subprocess.run(['ollama', 'pull', 'llama3.1:8b-instruct-q4_0'], #mistral:7b-instruct-q4_0
#                              capture_output=True, text=True)
# if model_result.returncode == 0:
#     print("✅ Mistral model ready!")
# else:
#     print("⚠️ Trying alternative model...")
#     !ollama pull tinyllama
#     print("✅ TinyLlama model ready!")

# # 5. Test OpenAI compatibility endpoint
# print("\n🧪 Testing OpenAI compatibility...")

# # Test if Ollama responds to OpenAI-style requests
# test_payload = '''
# {
#   "model": "mistral:7b-instruct-q4_0",
#   "messages": [{"role": "user", "content": "Hi"}],
#   "stream": false
# }
# '''

# # Test the OpenAI endpoint
# test_result = subprocess.run([
#     'curl', '-X', 'POST',
#     'http://localhost:11434/v1/chat/completions',
#     '-H', 'Content-Type: application/json',
#     '-d', test_payload,
#     '-s', '-w', '\nHTTP_CODE:%{http_code}'
# ], capture_output=True, text=True)

# if "HTTP_CODE:200" in test_result.stdout:
#     print("✅ OpenAI compatibility mode is working!")
# elif "HTTP_CODE:404" in test_result.stdout:
#     print("⚠️ OpenAI endpoint not found. Testing Ollama native endpoint...")

#     # Test native Ollama endpoint
#     native_test = subprocess.run([
#         'curl', '-X', 'POST',
#         'http://localhost:11434/api/generate',
#         '-H', 'Content-Type: application/json',
#         '-d', '{"model": "mistral:7b-instruct-q4_0", "prompt": "Hi", "stream": false}',
#         '-s', '-w', '\nHTTP_CODE:%{http_code}'
#     ], capture_output=True, text=True)

#     if "HTTP_CODE:200" in native_test.stdout:
#         print("✅ Native Ollama API is working!")
#         print("⚠️ Note: You may need to use Ollama-specific Spring AI configuration")
#     else:
#         print(f"❌ API test failed: {native_test.stdout[-50:]}")

# # 6. Install CloudFlared
# print("\n🌐 Setting up CloudFlare tunnel...")
# if not os.path.exists('./cloudflared-linux-amd64'):
#     !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
#     !chmod +x cloudflared-linux-amd64
# print("✅ CloudFlared ready!")

# # 7. Start tunnel
# print("\n🔗 Creating public tunnel...")
# print("="*60)

# # Start tunnel in background
# tunnel_process = subprocess.Popen([
#     './cloudflared-linux-amd64', 'tunnel',
#     '--url', 'http://localhost:11434'
# ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# # Capture URL
# public_url = None
# start_time = time.time()

# for line in tunnel_process.stdout:
#     # Check for URL
#     if 'trycloudflare.com' in line:
#         url_match = re.search(r'https://[a-zA-Z0-9\-]+\.trycloudflare\.com', line)
#         if url_match and not public_url:
#             public_url = url_match.group(0).strip()
#             break

#     # Timeout after 30 seconds
#     if time.time() - start_time > 30:
#         break

# if public_url:
#     print("\n" + "🎉"*25)
#     print("SUCCESS! Ollama is publicly accessible!")
#     print("🎉"*25)

#     # Clean URL display
#     print(f"\n📋 COPY THIS URL (no spaces):")
#     print("="*70)
#     print(public_url)
#     print("="*70)

#     # Test the public URL
#     print("\n🧪 Testing public tunnel...")

#     # Test root endpoint
#     public_test = subprocess.run([
#         'curl', '-s', '-o', '/dev/null', '-w', '%{http_code}',
#         public_url
#     ], capture_output=True, text=True, timeout=10)

#     print(f"✅ Tunnel response code: {public_test.stdout}")

#     # Test OpenAI endpoint through tunnel
#     openai_test = subprocess.run([
#         'curl', '-X', 'POST',
#         f'{public_url}/v1/chat/completions',
#         '-H', 'Content-Type: application/json',
#         '-d', test_payload,
#         '-s', '-w', '\nHTTP_CODE:%{http_code}'
#     ], capture_output=True, text=True, timeout=10)

#     if "HTTP_CODE:200" in openai_test.stdout:
#         print("✅ OpenAI endpoint accessible through tunnel!")
#         config_type = "OpenAI"
#     elif "HTTP_CODE:404" in openai_test.stdout:
#         print("⚠️ OpenAI endpoint returns 404")
#         print("ℹ️ This Ollama version might not support OpenAI compatibility")
#         config_type = "Ollama"
#     else:
#         print(f"⚠️ OpenAI endpoint test result: {openai_test.stdout[-50:]}")
#         config_type = "Unknown"

#     # Show appropriate configuration
#     print("\n📝 Your application.properties configuration:")
#     print("-"*70)

#     if config_type == "OpenAI":
#         print(f"spring.ai.openai.base-url={public_url}")
#         print("spring.ai.openai.api-key=ollama")
#         print("spring.ai.openai.chat.options.model=mistral:7b-instruct-q4_0")
#         print("spring.ai.openai.chat.options.temperature=0.1")
#     else:
#         print("# ⚠️ If OpenAI compatibility doesn't work, try:")
#         print("# Option 1: Use Ollama-specific configuration")
#         print(f"spring.ai.ollama.base-url={public_url}")
#         print("spring.ai.ollama.chat.options.model=mistral:7b-instruct-q4_0")
#         print("")
#         print("# Option 2: Keep OpenAI config but check if model exists")
#         print(f"spring.ai.openai.base-url={public_url}")
#         print("spring.ai.openai.api-key=ollama")
#         print("spring.ai.openai.chat.options.model=mistral:7b-instruct-q4_0")

#     print("-"*70)

#     # List available models
#     print("\n📦 Available models:")
#     !curl -s {public_url}/api/tags | python -m json.tool 2>/dev/null || echo "Could not list models"

# else:
#     print("\n❌ Could not capture tunnel URL!")
#     print("Checking tunnel output manually...")
#     !ps aux | grep cloudflared
#     print("\nTry running this command manually:")
#     print("!./cloudflared-linux-amd64 tunnel --url http://localhost:11434")

# print("\n" + "="*60)
# print("⚠️ IMPORTANT NOTES:")
# print("1. KEEP THIS NOTEBOOK RUNNING")
# print("2. If you get 404, ensure model name matches exactly")
# print("3. Try 'mistral:7b' instead of 'mistral:7b-instruct-q4_0' if needed")
# print("="*60)

# # Keep alive
# print(f"\n💚 Started at {time.strftime('%H:%M:%S')} - Keep tab open!")

# while True:
#     time.sleep(3600)

🚀 Starting Ollama with OpenAI Compatibility Mode...

📦 Installing Ollama...
✅ Ollama installed!

🔧 Starting Ollama server...
🔍 Verifying Ollama...
✅ Ollama is running!
NAME    ID    SIZE    MODIFIED 

📥 Downloading model (2-5 minutes)...
✅ Mistral model ready!

🧪 Testing OpenAI compatibility...
⚠️ OpenAI endpoint not found. Testing Ollama native endpoint...
❌ API test failed: istral:7b-instruct-q4_0' not found"}
HTTP_CODE:404

🌐 Setting up CloudFlare tunnel...
✅ CloudFlared ready!

🔗 Creating public tunnel...

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
SUCCESS! Ollama is publicly accessible!
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

📋 COPY THIS URL (no spaces):
https://walnut-races-campaign-indexes.trycloudflare.com

🧪 Testing public tunnel...
✅ Tunnel response code: 000
⚠️ OpenAI endpoint test result: 
HTTP_CODE:000

📝 Your application.properties configuration:
----------------------------------------------------------------------
# ⚠️ If OpenAI compatibility doesn't work, try:
# Option 1: Use Ollama-specific config

KeyboardInterrupt: 

In [ ]:
# ============================================
# FIXED: OLLAMA WITH PROPER TUNNEL WAIT
# ============================================

# First Uncomment the previous cell and run and then comment it and run this cell so that model loading will not have issues

import subprocess
import threading
import time
import re
import os

print("🚀 Starting Ollama with OpenAI Compatibility Mode...")
print("="*60)

# 1. Install Ollama
print("\n📦 Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1
print("✅ Ollama installed!")

# 2. Start Ollama
print("\n🔧 Starting Ollama server...")
os.environ['OLLAMA_ORIGINS'] = '*'
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'

def run_ollama():
    env = os.environ.copy()
    env.update({'OLLAMA_ORIGINS': '*', 'OLLAMA_HOST': '0.0.0.0:11434'})
    subprocess.run(['ollama', 'serve'], env=env,
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

threading.Thread(target=run_ollama, daemon=True).start()
time.sleep(7)

# 3. Verify Ollama
result = subprocess.run(['curl', '-s', 'http://localhost:11434'],
                       capture_output=True, text=True)
if "Ollama" in result.stdout:
    print("✅ Ollama is running!")
else:
    print("⚠️ Ollama might still be starting...")

# 4. Check available models
print("\n📦 Available models:")
!ollama list

# 5. Use llama3.1:8b-instruct (better for travel planning)
model_name = "llama3.1:8b-instruct-q4_0"

# Check if model exists
model_check = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
if model_name not in model_check.stdout:
    print(f"\n📥 Downloading {model_name} (this takes 5-10 minutes)...")
    !ollama pull llama3.1:8b-instruct
    print("✅ Model ready!")
else:
    print(f"✅ {model_name} already available!")

# 6. Test OpenAI compatibility
print("\n🧪 Testing OpenAI compatibility...")
test_payload = f'{{"model":"{model_name}","messages":[{{"role":"user","content":"Hi"}}],"stream":false}}'
test_result = subprocess.run([
    'curl', '-X', 'POST', 'http://localhost:11434/v1/chat/completions',
    '-H', 'Content-Type: application/json',
    '-d', test_payload, '-s', '-w', '\nHTTP:%{http_code}'
], capture_output=True, text=True, timeout=30)

if "HTTP:200" in test_result.stdout:
    print("✅ OpenAI compatibility working!")
else:
    print(f"⚠️ OpenAI endpoint returned: {test_result.stdout[-50:]}")

# 7. Install CloudFlared
print("\n🌐 Setting up CloudFlare tunnel...")
if not os.path.exists('./cloudflared-linux-amd64'):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64
print("✅ CloudFlared ready!")

# 8. Start tunnel with better capture
print("\n🔗 Starting CloudFlare tunnel (takes 15-30 seconds)...")
print("="*70)

# Kill any existing tunnels
!pkill -9 cloudflared 2>/dev/null

# Start new tunnel and save output
tunnel_log = '/tmp/tunnel.log'
subprocess.Popen(
    ['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:11434'],
    stdout=open(tunnel_log, 'w'),
    stderr=subprocess.STDOUT
)

# Wait for tunnel to establish
public_url = None
for attempt in range(60):  # Try for 60 seconds
    time.sleep(1)

    try:
        with open(tunnel_log, 'r') as f:
            content = f.read()
            urls = re.findall(r'https://[a-zA-Z0-9\-]+\.trycloudflare\.com', content)
            if urls:
                public_url = urls[0].strip()
                break
    except:
        pass

    if attempt % 5 == 0 and attempt > 0:
        print(f"⏳ Waiting for tunnel... {attempt}s")

if not public_url:
    print("\n❌ Could not capture URL. Checking logs...")
    !cat /tmp/tunnel.log
    print("\n⚠️ Try running this cell again")
else:
    print(f"\n🎉 Tunnel established!\n")

    # Wait a bit more for tunnel to stabilize
    print("⏳ Waiting for tunnel to stabilize (10s)...")
    time.sleep(10)

    # Test the tunnel
    print("\n🧪 Testing tunnel connectivity...")

    # Test 1: Root endpoint
    root_test = subprocess.run(
        ['curl', '-s', '-m', '10', '-o', '/dev/null', '-w', '%{http_code}', public_url],
        capture_output=True, text=True
    )
    print(f"  Root endpoint: HTTP {root_test.stdout}")

    # Test 2: API tags
    tags_test = subprocess.run(
        ['curl', '-s', '-m', '10', '-o', '/dev/null', '-w', '%{http_code}',
         f'{public_url}/api/tags'],
        capture_output=True, text=True
    )
    print(f"  /api/tags: HTTP {tags_test.stdout}")

    # Test 3: OpenAI endpoint
    openai_test = subprocess.run([
        'curl', '-X', 'POST', '-s', '-m', '30',
        f'{public_url}/v1/chat/completions',
        '-H', 'Content-Type: application/json',
        '-d', test_payload,
        '-w', '\nHTTP:%{http_code}'
    ], capture_output=True, text=True)

    openai_status = "200" in openai_test.stdout or "HTTP:200" in openai_test.stdout
    print(f"  /v1/chat/completions: {'✅ Working' if openai_status else '❌ Failed'}")

    # Display configuration
    print("\n" + "="*70)
    print("📋 YOUR TUNNEL URL:")
    print("="*70)
    print(public_url)
    print("="*70)

    print("\n📝 COPY THIS TO application.properties:")
    print("-"*70)
    print(f"spring.ai.openai.base-url={public_url}")
    print("spring.ai.openai.api-key=ollama")
    print(f"spring.ai.openai.chat.options.model={model_name}")
    print("spring.ai.openai.chat.options.temperature=0.7")
    print("spring.ai.openai.chat.options.max-tokens=1024")
    print("spring.ai.openai.chat.options.top-p=0.9")
    print("-"*70)

    # List models via tunnel
    print("\n📦 Verifying models via tunnel:")
    !curl -s {public_url}/api/tags | python3 -m json.tool 2>/dev/null || echo "⚠️ Could not list models"

    print("\n" + "="*70)
    print("⚠️ IMPORTANT:")
    print("  1. Keep this notebook tab open")
    print("  2. Tunnel URL changes each session")
    print("  3. Session lasts ~12 hours on free Colab")
    print("="*70)

# Keep alive with periodic status
print(f"\n💚 Running since {time.strftime('%H:%M:%S')}")
print("💡 Notebook will print status every 30 minutes\n")

import datetime
counter = 0
while True:
    time.sleep(1800)  # 30 minutes
    counter += 1
    print(f"💚 Still alive [{datetime.datetime.now().strftime('%H:%M:%S')}] - {counter*30} min uptime")

🚀 Starting Ollama with OpenAI Compatibility Mode...

📦 Installing Ollama...
✅ Ollama installed!

🔧 Starting Ollama server...
✅ Ollama is running!

📦 Available models:
NAME                         ID              SIZE      MODIFIED      
llama3.1:8b-instruct-q4_0    42182419e950    4.7 GB    2 minutes ago    
✅ llama3.1:8b-instruct-q4_0 already available!

🧪 Testing OpenAI compatibility...
✅ OpenAI compatibility working!

🌐 Setting up CloudFlare tunnel...
✅ CloudFlared ready!

🔗 Starting CloudFlare tunnel (takes 15-30 seconds)...

🎉 Tunnel established!

⏳ Waiting for tunnel to stabilize (10s)...

🧪 Testing tunnel connectivity...
  Root endpoint: HTTP 200
  /api/tags: HTTP 200
  /v1/chat/completions: ✅ Working

📋 YOUR TUNNEL URL:
https://relate-susan-pepper-manner.trycloudflare.com

📝 COPY THIS TO application.properties:
----------------------------------------------------------------------
spring.ai.openai.base-url=https://relate-susan-pepper-manner.trycloudflare.com
spring.ai.openai.ap

KeyboardInterrupt: 